In [16]:
import pandas as pd

from epitools import get_data

In [32]:
country = "France"
year = 2022
geography_id = "department_id"

country = "Chile"
year = 2021
geography_id = "region_id"

In [34]:
df = pd.read_csv(f"data_output/{country}/{year}_first_round.csv.gz")
df_location = pd.read_csv(f"data_output/{country}/{year}_first_round_location.csv.gz")
df = pd.merge(df, df_location[["polling_id", geography_id]], on="polling_id")
df["type"] = df["candidate"].apply(lambda x: "ABSTENTION" if x == "ABSTENTION" else "OTHER")

df.head()

,polling_id,candidate,value,rate,rank,flag_candidates,region_id,type
0,ALGARROBO-ALGARROBO-1.0-M-1M,EDUARDO ARTES BRICHETTI,1.0,0.006024,7.0,1,5,OTHER
1,ALGARROBO-ALGARROBO-1.0-M-1M,FRANCO PARISI FERNANDEZ,14.0,0.084337,5.0,1,5,OTHER
2,ALGARROBO-ALGARROBO-1.0-M-1M,GABRIEL BORIC FONT,29.0,0.174699,3.0,1,5,OTHER
3,ALGARROBO-ALGARROBO-1.0-M-1M,JOSE ANTONIO KAST RIST,53.0,0.319277,1.0,1,5,OTHER
4,ALGARROBO-ALGARROBO-1.0-M-1M,MARCO ENRIQUEZ-OMINAMI GUMUCIO,10.0,0.060241,6.0,1,5,OTHER


In [35]:
dd = df.groupby([geography_id, "type"]).agg({"value": "sum"})
dd["rate"] = dd.groupby(level=[0], group_keys=False).apply(lambda x: x/x.sum())
dd = dd.reset_index()
dd = dd[dd["type"] == "OTHER"]
dd

,region_id,type,value,rate
1,1,OTHER,110354.0,0.424199
3,2,OTHER,203838.0,0.423076
5,3,OTHER,107185.0,0.441492
7,4,OTHER,267015.0,0.428197
9,5,OTHER,802976.0,0.497929
11,6,OTHER,368873.0,0.472730
13,7,OTHER,409512.0,0.458968
15,8,OTHER,620395.0,0.463809
17,9,OTHER,400139.0,0.450807
19,10,OTHER,330951.0,0.443424


In [39]:
df_ep = get_data(
    country=country,
    aggregation=geography_id,
    election="first_round",
    year=year
)

if country == "Chile":
    df_ep[geography_id] = df_ep[geography_id].astype(int)

In [41]:
data = pd.merge(df_ep, dd, on=geography_id)
if country == "France":
    data = data[~data[geography_id].str.contains("Z")]
print(data.shape)

data.corr()

(16, 8)


C:\Users\cnava\AppData\Local\Temp\ipykernel_9380\3581127434.py:6: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.corr()


,region_id,epi,epi_between,epi_within,year,value,rate
region_id,1.000000,-0.343454,-0.038049,-0.445462,NaN,0.171103,0.221348
epi,-0.343454,1.000000,0.739882,0.473196,NaN,0.351612,0.390648
epi_between,-0.038049,0.739882,1.000000,-0.242543,NaN,-0.112475,0.192363
epi_within,-0.445462,0.473196,-0.242543,1.000000,NaN,0.654341,0.311444
year,NaN,NaN,NaN,NaN,NaN,NaN,NaN
value,0.171103,0.351612,-0.112475,0.654341,NaN,1.000000,0.678953
rate,0.221348,0.390648,0.192363,0.311444,NaN,0.678953,1.000000
